<a href="https://colab.research.google.com/github/alexanderportilla96/Sy2025II/blob/main/Correccion_de_Streamlit_de_Solucion_parcial_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install streamlit control matplotlib -q
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared
!mv cloudflared /usr/local/bin/cloudflared
!pip install yt-dlp -q
!apt-get install ffmpeg -y
!mkdir -p pages

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.3/578.3 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 62.1 MB/s eta 0:00:00
--2025-12-13 03:21:11--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64 [following]
--2025-12-13 03:21:12--  https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/955e9d1b-ac5e-4188-8867-e5f53958a8fe?sp=r&sv=201

In [18]:
%%writefile 0_Inicio.py
import streamlit as st

st.set_page_config(
    page_title="Parcial 2 - Señales y Sistemas 2025-II",
    page_icon="",
    layout="centered"
)

st.markdown("<h1 style='text-align: center; color: #004488;'>Parcial 2 - Señales y Sistemas 2025-II</h1>", unsafe_allow_html=True)
st.markdown("---")

st.markdown("""
Aplicación que presenta, el solucionario **Parcial 2 (2025-II)**, implementada con paneles interactivos de Streamlit.

### 📌 Contenido

#### 1️. Demodulación en Amplitud (DSB-SC)
- Modulación y demodulación síncrona.
- **Filtrado pasa bajas implementado con FFT (Transformada Rápida de Fourier)**.

#### 2️. Sistema Masa-Resorte y Equivalente Eléctrico
- Simulación de respuestas al **Impulso, Escalón y Rampa**.
- Análisis de amortiguamiento.

---
### ESTUDIANTE: Arthur Alexander Portilla C.C.1088596640
- **Curso:** Señales y Sistemas
""")

st.sidebar.success("Selecciona un punto en el menú lateral.")

Overwriting 0_Inicio.py


In [19]:
%%writefile pages/1_Demodulacion.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft, fftfreq, fftshift
from scipy.io import wavfile
import os
import subprocess
import yt_dlp

st.set_page_config(page_title="Punto 1: Demodulación DSB-SC", layout="wide")

st.markdown("<h1 style='text-align: center;'>📻 Punto 1: Demodulación de Amplitud (DSB-SC)</h1>", unsafe_allow_html=True)
st.markdown("---")

# --- FUNCIONES DE SOPORTE ---
def load_audio_from_youtube(url, duration_s=5, start_s=0):
    temp_wav = 'temp_audio.wav'
    cropped_wav = 'cropped.wav'
    try:
        ydl_opts = {'format': 'bestaudio/best', 'outtmpl': 'temp_audio', 'quiet': True,
                    'postprocessors': [{'key': 'FFmpegExtractAudio','preferredcodec': 'wav'}]}
        if os.path.exists(temp_wav): os.remove(temp_wav)

        # El archivo que descarga youtube-dl es temp_audio.wav
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])

        # Corregir el nombre del archivo si es necesario, o asegurar que se llame temp_audio.wav
        # Si yt-dlp guarda como temp_audio.mp3, esto fallará. Ajustamos para ser robustos.
        # Buscamos el archivo generado que termine en .wav
        downloaded_file = next((f for f in os.listdir('.') if f.startswith('temp_audio') and f.endswith('.wav')), 'temp_audio.wav')


        subprocess.run([
            'ffmpeg', '-i', downloaded_file, '-ss', str(start_s), '-t', str(duration_s),
            '-acodec', 'pcm_s16le', '-ar', '44100', '-ac', '1', cropped_wav, '-y'
        ], check=True, capture_output=True)

        fs, data = wavfile.read(cropped_wav)
        if data.ndim > 1: data = data.mean(axis=1)
        data = data / np.max(np.abs(data)) # Normalizar
        return data, fs
    except Exception as e:
        st.error(f"❌ Error descargando audio. Asegúrate de que la URL es correcta y ffmpeg está disponible: {e}")
        return None, None

def plot_spectrum(sig, fs, title, ax):
    N = len(sig)
    Y = fft(sig)
    freqs = fftfreq(N, 1/fs)
    Y_shift = fftshift(Y)
    freq_shift = fftshift(freqs)
    ax.plot(freq_shift, np.abs(Y_shift))
    ax.set_title(title)
    ax.set_xlabel("Frecuencia (Hz)")
    ax.set_xlim(-20000, 20000)
    ax.grid(True, alpha=0.3)

def fft_lowpass_filter(signal, fs, cutoff):
    """Implementación del filtro pasa bajas usando FFT (Requerimiento del parcial)"""
    N = len(signal)
    F = fft(signal)
    freqs = fftfreq(N, 1/fs)

    # Máscara ideal: pasa si |f| < cutoff
    mask = np.abs(freqs) < cutoff
    F_filtered = F * mask

    return ifft(F_filtered).real

# --- INTERFAZ Y PROCESAMIENTO ---
st.sidebar.header("Parámetros de Demodulación")
# Se utiliza una URL de ejemplo genérica, el usuario debe cambiarla
url = st.sidebar.text_input("URL YouTube", "https://www.youtube.com/watch?v=dQw4w9WgXcQ")
start_time = st.sidebar.slider("Inicio (s)", 0, 60, 10)
fc = st.sidebar.slider("Frecuencia Portadora (fc)", 3000, 15000, 10000)
cutoff = st.sidebar.slider("Corte Filtro LPF (Hz)", 1000, 8000, 4000)

if st.sidebar.button("Simular Demodulación"):
    with st.spinner("Procesando audio y simulando..."):
        m_t, fs = load_audio_from_youtube(url, duration_s=5, start_s=start_time)

        if m_t is not None:
            t = np.linspace(0, len(m_t)/fs, len(m_t), endpoint=False)

            # Etapa 1: Modulación DSB-SC
            carrier = np.cos(2 * np.pi * fc * t)
            x_t = m_t * carrier

            # Etapa 2: Mezcla Sincrónica (Demodulación)
            y_t = x_t * carrier

            # Etapa 3: Filtrado (Vía FFT)
            m_recuperada = fft_lowpass_filter(y_t, fs, cutoff)

            # Etapa 4: Escalamiento (x2 para recuperar amplitud Ac=1)
            m_final = m_recuperada * 2

            # --- VISUALIZACIÓN ---
            st.subheader("1. Señal de Mensaje Original m(t)")
            colA, colB = st.columns(2)
            with colA: figA, axA = plt.subplots(figsize=(6, 3)); axA.plot(t[:1000], m_t[:1000]); axA.set_title("Tiempo (Zoom)"); st.pyplot(figA)
            with colB: figB, axB = plt.subplots(figsize=(6, 3)); plot_spectrum(m_t, fs, "Espectro Original", axB); st.pyplot(figB)
            st.audio(m_t, sample_rate=fs)

            st.subheader("2. Señal Modulada x(t)")
            colC, colD = st.columns(2)
            with colC: figC, axC = plt.subplots(figsize=(6, 3)); axC.plot(t[:200], x_t[:200], color='orange'); axC.set_title("Tiempo (Zoom)"); st.pyplot(figC)

            # CORRECCIÓN DE TYPO: Se asegura que se use axD para la columna D (Espectro Modulado)


            with colD: figD, axD = plt.subplots(figsize=(6, 3)); plot_spectrum(x_t, fs, "Espectro Modulado", axD); st.pyplot(figD)

            st.subheader("3. Salida del Mezclador y(t) (Antes del Filtro)")
            colE, colF = st.columns(2)
            with colE: figE, axE = plt.subplots(figsize=(6, 3)); axE.plot(t[:200], y_t[:200], color='green'); axE.set_title("Tiempo (Zoom)"); st.pyplot(figE)
            with colF: figF, axF = plt.subplots(figsize=(6, 3)); plot_spectrum(y_t, fs, "Espectro (Banda base + 2fc)", axF); st.pyplot(figF)

            st.subheader(f"4. Señal Recuperada m'(t) (Filtro FFT corte {cutoff}Hz)")
            colG, colH = st.columns(2)
            with colG: figG, axG = plt.subplots(figsize=(6, 3)); axG.plot(t[:1000], m_final[:1000], color='purple'); axG.set_title("Tiempo (Zoom)"); st.pyplot(figG)
            with colH: figH, axH = plt.subplots(figsize=(6, 3)); plot_spectrum(m_final, fs, "Espectro Recuperado", axH); st.pyplot(figH)
            st.audio(m_final, sample_rate=fs)

Overwriting pages/1_Demodulacion.py


In [20]:
%%writefile pages/2_Sistema.py
import streamlit as st
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
from control import matlab

#
# 1. PARAMETRIZACIÓN DEL SISTEMA
#
# Definimos las propiedades físicas del sistema mecánico
m = 1.0   # Masa (kg) - Representa la inercia
c = 0.5   # Amortiguador (Ns/m) - Representa la fricción/disipación
k = 4.0   # Resorte (N/m) - Representa la rigidez/restitución

#
# 2. MODELADO (Función de Transferencia)
#
# G(s) = 1 / (ms^2 + cs + k)
num = [1]
den = [m, c, k]
sistema = signal.TransferFunction(num, den)

# Calculamos parámetros clave para el reporte
wn = np.sqrt(k/m)            # Frecuencia natural
zeta = c / (2 * np.sqrt(k*m)) # Factor de amortiguamiento

#
# 3. SIMULACIÓN DE SEÑALES
#
# A. Respuesta al Escalón (Fuerza constante de 1N)
t_step, y_step = signal.step(sistema)

# B. Polos del sistema (Estabilidad)
polos = sistema.poles

# C. Diagrama de Bode (Respuesta en Frecuencia)
w, mag, phase = signal.bode(sistema)

# D. Respuesta al Impulso (Golpe seco)
t_imp, y_imp = signal.impulse(sistema)

#
# 4. GRAFICACIÓN (Dashboard de Ingeniería)
#
plt.style.use('seaborn-v0_8-darkgrid')
fig, ax = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle(f'Análisis Dinámico: Masa-Resorte-Amortiguador\n(m={m}, c={c}, k={k})', fontsize=16, weight='bold')

# Gráfica 1: Escalón
ax[0, 0].plot(t_step, y_step, 'b-', linewidth=2)
ax[0, 0].set_title("1. Respuesta al Escalón (Step Response)")
ax[0, 0].set_ylabel("Posición (m)")
ax[0, 0].axhline(1/k, color='r', linestyle='--', label=f'Ref: {1/k}m')
ax[0, 0].legend()

# Gráfica 2: Polos
ax[0, 1].scatter(np.real(polos), np.imag(polos), marker='x', s=100, c='red')
ax[0, 1].set_title("2. Ubicación de Polos (Plano S)")
ax[0, 1].set_xlabel("Real (Amortiguamiento)")
ax[0, 1].set_ylabel("Imaginario (Oscilación)")
ax[0, 1].axvline(0, color='k'); ax[0, 1].axhline(0, color='k')

# Gráfica 3: Bode
ax[1, 0].semilogx(w, mag, 'g-', linewidth=2)
ax[1, 0].set_title("3. Bode (Magnitud)")
ax[1, 0].set_ylabel("Ganancia (dB)")
ax[1, 0].axvline(wn, color='orange', linestyle='--', label='Frec. Natural')
ax[1, 0].legend()

# Gráfica 4: Impulso
ax[1, 1].plot(t_imp, y_imp, 'm-', linewidth=2)
ax[1, 1].set_title("4. Respuesta al Impulso (Impulse Response)")
ax[1, 1].set_xlabel("Tiempo (s)")

plt.tight_layout()
plt.show()


# 5. REPORTE TÉCNICO AUTOMÁTICO (EXPLICACIÓN)


print("\n" + "="*60)
print("             REPORTE DE INGENIERÍA DE CONTROL")
print("="*60)

print(f"\n--- 1. ANÁLISIS DE LA GRÁFICA 1 (ESCALÓN) ---")
print(f"Interpretación: Simulamos aplicar una fuerza constante de 1 Newton.")
print(f"- Estado Estable: La posición final es {1/k} metros (calculado como 1/k).")
if zeta < 1:
    print("- Transitorio: Se observa un 'sobrepico' (la masa sube más de la cuenta y regresa).")
    print("  Esto indica que la inercia supera momentáneamente a la fricción.")
else:
    print("- Transitorio: El sistema sube suavemente sin oscilar (Sobre-amortiguado).")

print(f"\n--- 2. ANÁLISIS DE LA GRÁFICA 2 (POLOS) ---")
print(f"- Ubicación: {np.round(polos, 3)}")
if np.all(np.real(polos) < 0):
    print("- Estabilidad: ESTABLE (Están a la izquierda). La energía se disipa.")
else:
    print("- Estabilidad: INESTABLE (Peligro). El sistema oscilaría hasta romperse.")
if np.iscomplex(polos).any():
    print("- Dinámica: Tienen parte imaginaria, lo que confirma matemáticamente que el sistema OSCILA.")

print(f"\n--- 3. ANÁLISIS DE LA GRÁFICA 3 (BODE) ---")
print(f"- Frecuencia Natural (wn): {round(wn, 2)} rad/s.")
print("- Interpretación: Si vibras el sistema a esta frecuencia, verás el pico de resonancia.")
print("  A altas frecuencias (derecha), la gráfica cae, indicando que la masa no puede")
print("  seguir movimientos muy rápidos (filtro pasa bajas).")

print(f"\n--- 4. ANÁLISIS DE LA GRÁFICA 4 (IMPULSO) ---")
print("- Interpretación: Simulamos un golpe seco (martillazo).")
print("- Comportamiento: La masa sale disparada (velocidad instantánea) y luego")
print("  el amortiguador disipa la energía hasta que se detiene en cero.")
print("="*60)

Overwriting pages/2_Sistema.py


In [12]:
%%writefile pages/2_Sistema.py
import streamlit as st
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
from control import matlab

st.set_page_config(page_title="Punto 2: Sistema Dinámico", layout="wide")

st.markdown("<h1 style='text-align: center;'>⚙️ Punto 2: Sistema Mecánico y Equivalente Eléctrico</h1>", unsafe_allow_html=True)
st.markdown("---")

# --- MODELADO TEÓRICO ---
st.markdown("### 📝 Función de Transferencia (Lazo Abierto $G(s)$)")
st.markdown("El sistema masa-resorte-amortiguador tiene la ecuación diferencial: $m\\ddot{y} + c\\dot{y} + ky = F_E$")
st.latex(r"G(s) = H_{mec}(s) = \frac{Y(s)}{F(s)} = \frac{1/m}{s^2 + \frac{c}{m}s + \frac{k}{m}}")

# Muestra el sistema mecánico y el equivalente.
st.image("https://i.imgur.com/8Qj8n60.png", caption="Sistema Masa-Resorte-Amortiguador y Circuito Equivalente RLC.")

# --- CONTROLES ---
st.sidebar.header("Parámetros del Sistema (m, k, c)")
m = st.sidebar.number_input("Masa (m) [kg]", value=1.0, step=0.1)
k = st.sidebar.number_input("Resorte (k) [N/m]", value=10.0, step=1.0)

modo = st.sidebar.radio("Modo de Amortiguamiento", ["Subamortiguado", "Crítico", "Sobreamortiguado"])

wn = np.sqrt(k/m)
cc = 2 * np.sqrt(k*m)

if modo == "Subamortiguado":
    c = st.sidebar.slider("Coeficiente c", 0.01*cc, cc*0.99, cc*0.3)
elif modo == "Crítico":
    c = cc
    st.sidebar.info(f"c fijado a {c:.2f} para amortiguamiento crítico")
elif modo == "Sobreamortiguado":
    c = st.sidebar.slider("Coeficiente c", cc*1.01, cc*5.0, cc*1.5)

zeta = c / (2 * np.sqrt(k*m))

# --- FUNCIÓN DE TRANSFERENCIA ---
# 1. Lazo Abierto (Open-Loop - OL)
num_ol = [1/m]
den_ol = [1, c/m, k/m]
sys_ol = signal.TransferFunction(num_ol, den_ol)

# 2. Lazo Cerrado (Closed-Loop - CL): Asumimos retroalimentación unitaria H=1: G_CL = G / (1 + G)
# G(s) = (1/m) / (s^2 + (c/m)s + k/m)
# G_CL(s) = (1/m) / (s^2 + (c/m)s + k/m + 1/m)
num_lc = num_ol
den_lc = [1, c/m, (k + 1) / m]
sys_lc = signal.TransferFunction(num_lc, den_lc)

# --- CÁLCULO DE EQUIVALENCIA ELÉCTRICA (No se modifica) ---
C_elec = 1.0
L_elec = (m/k) / C_elec
R_elec = (m/c) / C_elec if c != 0 else np.inf

st.markdown("### 📐 Parámetros del Sistema")
col1, col2, col3 = st.columns(3)
with col1: st.metric("Frecuencia Natural $\\omega_n$", f"{wn:.2f} rad/s")
with col2: st.metric("Factor de Amortiguamiento $\\zeta$", f"{zeta:.2f}")

st.markdown("#### **Equivalentes Eléctricos (asumiendo C=1F):**")
st.code(f"R = {R_elec:.3f} Ω | L = {L_elec:.3f} H | C = {C_elec:.1f} F")

# Tiempo
t = np.linspace(0, 15, 1000)

# Gráficas
st.markdown("---")
st.markdown("### 📊 Análisis del Comportamiento")
tab1, tab2, tab3 = st.tabs(["⚡ Respuestas Temporales", "🌀 Polos y Ceros", "📈 Diagrama de Bode"])

with tab1:
    st.subheader("Respuestas al Impulso, Escalón y Rampa (Lazo Abierto vs. Cerrado)")

    fig1, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))

    # 1. Impulso
    t_imp, y_imp_ol = signal.impulse(sys_ol, T=t)
    _, y_imp_lc = signal.impulse(sys_lc, T=t)
    ax1.plot(t_imp, y_imp_ol, 'r', label="Lazo Abierto (OL)"); ax1.plot(t_imp, y_imp_lc, 'r--', label="Lazo Cerrado (CL)")
    ax1.set_title("Respuesta al Impulso"); ax1.set_xlabel("Tiempo (s)"); ax1.grid(True); ax1.legend()

    # 2. Escalón
    t_step, y_step_ol = signal.step(sys_ol, T=t)
    _, y_step_lc = signal.step(sys_lc, T=t)
    ax2.plot(t_step, y_step_ol, 'b', label="Lazo Abierto (OL)"); ax2.plot(t_step, y_step_lc, 'b--', label="Lazo Cerrado (CL)")
    ax2.set_title("Respuesta al Escalón"); ax2.set_xlabel("Tiempo (s)"); ax2.grid(True); ax2.legend()

    # 3. Rampa: G(s)/s
    sys_ramp_ol = signal.TransferFunction(num_ol, np.polymul(den_ol, [1, 0]))
    t_ramp, y_ramp_ol = signal.step(sys_ramp_ol, T=t)

    sys_ramp_lc = signal.TransferFunction(num_lc, np.polymul(den_lc, [1, 0]))
    _, y_ramp_lc = signal.step(sys_ramp_lc, T=t)

    ax3.plot(t_ramp, y_ramp_ol, 'g', label="Lazo Abierto (OL)")
    ax3.plot(t_ramp, y_ramp_lc, 'g--', label="Lazo Cerrado (CL)")
    ax3.plot(t, t, 'k:', label="Rampa Entrada")
    ax3.set_title("Respuesta a la Rampa"); ax3.set_xlabel("Tiempo (s)"); ax3.grid(True); ax3.legend()

    st.pyplot(fig1)

with tab2:
    st.subheader("Mapa de Polos y Ceros")
    fig_pz, ax_pz = plt.subplots()

    # Polos Lazo Abierto
    ax_pz.scatter(np.real(sys_ol.poles), np.imag(sys_ol.poles), marker='x', s=100, color='blue', label='Polos OL')
    # Polos Lazo Cerrado
    ax_pz.scatter(np.real(sys_lc.poles), np.imag(sys_lc.poles), marker='o', s=100, color='red', facecolors='none', edgecolors='red', label='Polos CL')

    ax_pz.axhline(0, color='black', lw=1); ax_pz.axvline(0, color='black', lw=1)
    ax_pz.set_title("Mapa de Polos y Ceros"); ax_pz.set_xlabel("Eje Real"); ax_pz.set_ylabel("Eje Imaginario")
    ax_pz.grid(True); ax_pz.legend()
    st.pyplot(fig_pz)

with tab3:
    st.subheader("Diagrama de Bode (Lazo Abierto vs. Cerrado)")
    w_ol, mag_ol, phase_ol = signal.bode(sys_ol)
    w_lc, mag_lc, phase_lc = signal.bode(sys_lc)

    fig_bode, (ax_m, ax_p) = plt.subplots(2, 1, figsize=(8, 6), sharex=True)

    # Magnitud
    ax_m.semilogx(w_ol, mag_ol, label="Lazo Abierto"); ax_m.semilogx(w_lc, mag_lc, '--', label="Lazo Cerrado")
    ax_m.set_ylabel("Magnitud (dB)"); ax_m.grid(True); ax_m.legend()

    # Fase
    ax_p.semilogx(w_ol, phase_ol, label="Lazo Abierto"); ax_p.semilogx(w_lc, phase_lc, '--', label="Lazo Cerrado")
    ax_p.set_ylabel("Fase (deg)"); ax_p.set_xlabel("Frecuencia (rad/s)"); ax_p.grid(True)
    st.pyplot(fig_bode)

Overwriting pages/2_Sistema.py


In [13]:
%%writefile pages/2_Sistema.py
import streamlit as st
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
from control import matlab

import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

# ==========================================
# 1. DEFINICIÓN DE CLASES Y FUNCIONES
# ==========================================
def calcular_metricas(sistema, nombre_caso):
    """Calcula parámetros del sistema de 2do orden automáticamente"""
    # Polos para hallar wn y zeta
    polos = sistema.poles
    # Asumimos sistema de 2do orden dominante
    p_dom = polos[0]
    wn = np.abs(p_dom)
    zeta = -np.real(p_dom) / wn
    wd = wn * np.sqrt(1 - zeta**2) if zeta < 1 else 0

    # Métricas temporales (aproximaciones estándar)
    if zeta < 1:
        Tp = np.pi / wd if wd > 0 else np.inf
        Tr = (np.pi - np.arccos(zeta)) / wd if wd > 0 else np.inf
    else:
        Tp = np.inf # No hay pico real en sobreamortiguado
        Tr = 2.2 / wn # Aprox

    Ts = 4 / (zeta * wn) # Criterio del 2%

    print(f"--- {nombre_caso} ---")
    print(f"  Polos: {np.round(polos, 3)}")
    print(f"  Zeta (amortiguamiento): {zeta:.4f}")
    print(f"  Wn (Frec. Natural): {wn:.4f} rad/s")
    if zeta < 1: print(f"  Wd (Frec. Amortiguada): {wd:.4f} rad/s")
    print(f"  Tp (Tiempo Pico): {Tp:.4f} s")
    print(f"  Tr (Tiempo Subida): {Tr:.4f} s")
    print(f"  Ts (Tiempo Establecimiento): {Ts:.4f} s")
    print("-" * 30)
    return wn, zeta

def graficar_sistema(m, k, c, R_eq, titulo_caso):
    """
    Genera el dashboard de 5 gráficas para un caso específico.
    Incluye Lazo Abierto (OL) y Lazo Cerrado (CL).
    """
    # --- SISTEMA LAZO ABIERTO (G) ---
    # G(s) = 1 / (ms^2 + cs + k)
    sys_ol = signal.TransferFunction([1], [m, c, k])

    # --- SISTEMA LAZO CERRADO (T) ---
    # T(s) = G / (1+G) = 1 / (ms^2 + cs + (k+1))
    # Feedback unitario negativo aumenta la "rigidez" aparente en 1 unidad
    sys_cl = signal.TransferFunction([1], [m, c, k + 1])

    # Cálculo de métricas para imprimir en consola
    print(f"\n>>> ANÁLISIS: {titulo_caso}")
    print(f"Equivalentes Eléctricos: R={R_eq:.2f}Ω, L=2.5H, C=0.1F")
    calcular_metricas(sys_ol, "Lazo Abierto")
    calcular_metricas(sys_cl, "Lazo Cerrado")

    # Vectores de tiempo
    t_step, y_step = signal.step(sys_ol)
    t_imp, y_imp = signal.impulse(sys_ol)

    # Rampa: La respuesta a rampa es la integral del escalón o lsim con t
    t_ramp = np.linspace(0, t_step[-1], 1000)
    u_ramp = t_ramp
    tout_ramp, y_ramp, _ = signal.lsim(sys_ol, u_ramp, t_ramp)

    # --- GRAFICACIÓN ---
    plt.style.use('seaborn-v0_8-darkgrid')
    fig = plt.figure(figsize=(16, 10))
    fig.suptitle(f'{titulo_caso}\n(Mec: m={m}, c={c}, k={k} | Elec: R={R_eq:.2f}$\Omega$)', fontsize=16, weight='bold')

    # Layout complejo con GridSpec
    gs = fig.add_gridspec(2, 3)

    # 1. Mapa de Polos y Ceros
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.scatter(np.real(sys_ol.poles), np.imag(sys_ol.poles), marker='x', s=100, color='b', label='Abierto')
    ax1.scatter(np.real(sys_cl.poles), np.imag(sys_cl.poles), marker='o', s=100, color='r', facecolors='none', label='Cerrado')
    ax1.axvline(0, color='k', lw=1); ax1.axhline(0, color='k', lw=1)
    ax1.set_title("1. Mapa de Polos")
    ax1.legend()

    # 2. Diagrama de Bode
    ax2 = fig.add_subplot(gs[0, 1])
    w, mag, phase = signal.bode(sys_ol)
    w_cl, mag_cl, phase_cl = signal.bode(sys_cl)
    ax2.semilogx(w, mag, 'b', label='Abierto')
    ax2.semilogx(w_cl, mag_cl, 'r--', label='Cerrado')
    ax2.set_title("2. Bode (Magnitud)")
    ax2.legend()

    # 3. Respuesta al Impulso
    ax3 = fig.add_subplot(gs[0, 2])
    t_imp_cl, y_imp_cl = signal.impulse(sys_cl)
    ax3.plot(t_imp, y_imp, 'b', label='Abierto')
    ax3.plot(t_imp_cl, y_imp_cl, 'r--', label='Cerrado')
    ax3.set_title("3. Respuesta al Impulso")
    ax3.legend()

    # 4. Respuesta al Escalón
    ax4 = fig.add_subplot(gs[1, 0])
    t_step_cl, y_step_cl = signal.step(sys_cl)
    ax4.plot(t_step, y_step, 'b', label=f'Abierto (Est={1/k:.2f})')
    ax4.plot(t_step_cl, y_step_cl, 'r--', label=f'Cerrado (Est={1/(k+1):.2f})')
    ax4.set_title("4. Respuesta al Escalón")
    ax4.legend()

    # 5. Respuesta a la Rampa
    ax5 = fig.add_subplot(gs[1, 1:])
    tout_ramp_cl, y_ramp_cl, _ = signal.lsim(sys_cl, u_ramp, t_ramp)
    ax5.plot(t_ramp, u_ramp, 'k:', label='Entrada Rampa', alpha=0.5)
    ax5.plot(t_ramp, y_ramp, 'b', label='Abierto')
    ax5.plot(t_ramp, y_ramp_cl, 'r--', label='Cerrado')
    ax5.set_title("5. Respuesta a la Rampa")
    ax5.legend()

    plt.tight_layout()
    plt.show()

# ==========================================
# 2. EJECUCIÓN DE LOS 3 CASOS
# ==========================================

# Constantes Fijas: m=1 kg, k=4 N/m (Wn = 2 rad/s)
# Equivalencia Eléctrica base: L = 2.5 H, C = 0.1 F

# CASO A: SUBAMORTIGUADO (Zeta = 0.25 -> c = 1)
# R = 10 / c = 10 Ohms
graficar_sistema(m=1, k=4, c=1, R_eq=10, titulo_caso="CASO 1: SUB-AMORTIGUADO (Zeta=0.25)")

# CASO B: CRÍTICAMENTE AMORTIGUADO (Zeta = 1.0 -> c = 4)
# R = 10 / c = 2.5 Ohms
graficar_sistema(m=1, k=4, c=4, R_eq=2.5, titulo_caso="CASO 2: CRÍTICO (Zeta=1.0)")

# CASO C: SOBREAMORTIGUADO (Zeta = 2.0 -> c = 8)
# R = 10 / c = 1.25 Ohms
graficar_sistema(m=1, k=4, c=8, R_eq=1.25, titulo_caso="CASO 3: SOBRE-AMORTIGUADO (Zeta=2.0)")

Overwriting pages/2_Sistema.py


In [16]:
%%writefile pages/2_Sistema.py
import streamlit as st
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
from control import matlab

st.set_page_config(page_title="Punto 2: Sistema Dinámico", layout="wide")

st.markdown("<h1 style='text-align: center;'>⚙️ Punto 2: Sistema Mecánico y Equivalente Eléctrico</h1>", unsafe_allow_html=True)
st.markdown("---")

# --- MODELADO TEÓRICO ---
st.markdown("### 📝 Función de Transferencia (Lazo Abierto $G(s)$)")
st.markdown("El sistema masa-resorte-amortiguador tiene la ecuación diferencial: $m\\ddot{y} + c\\dot{y} + ky = F_E$")
st.latex(r"G(s) = H_{mec}(s) = \frac{Y(s)}{F(s)} = \frac{1/m}{s^2 + \frac{c}{m}s + \frac{k}{m}}")


# --- CONTROLES ---
st.sidebar.header("Parámetros del Sistema (m, k, c)")
m = st.sidebar.number_input("Masa (m) [kg]", value=1.0, step=0.1)
k = st.sidebar.number_input("Resorte (k) [N/m]", value=10.0, step=1.0)

modo = st.sidebar.radio("Modo de Amortiguamiento", ["Subamortiguado", "Crítico", "Sobreamortiguado"])

wn = np.sqrt(k/m)
cc = 2 * np.sqrt(k*m)

if modo == "Subamortiguado":
    c = st.sidebar.slider("Coeficiente c", 0.01*cc, cc*0.99, cc*0.3)
elif modo == "Crítico":
    c = cc
    st.sidebar.info(f"c fijado a {c:.2f} para amortiguamiento crítico")
elif modo == "Sobreamortiguado":
    c = st.sidebar.slider("Coeficiente c", cc*1.01, cc*5.0, cc*1.5)

zeta = c / (2 * np.sqrt(k*m))

# --- FUNCIÓN DE TRANSFERENCIA ---
# 1. Lazo Abierto (Open-Loop - OL)
num_ol = [1/m]
den_ol = [1, c/m, k/m]
sys_ol = signal.TransferFunction(num_ol, den_ol)

# 2. Lazo Cerrado (Closed-Loop - CL): Asumimos retroalimentación unitaria H=1: G_CL = G / (1 + G)
# G(s) = (1/m) / (s^2 + (c/m)s + k/m)
# G_CL(s) = (1/m) / (s^2 + (c/m)s + k/m + 1/m)
num_lc = num_ol
den_lc = [1, c/m, (k + 1) / m]
sys_lc = signal.TransferFunction(num_lc, den_lc)

# --- CÁLCULO DE EQUIVALENCIA ELÉCTRICA (No se modifica) ---
C_elec = 1.0
L_elec = (m/k) / C_elec
R_elec = (m/c) / C_elec if c != 0 else np.inf

st.markdown("### 📐 Parámetros del Sistema")
col1, col2, col3 = st.columns(3)
with col1: st.metric("Frecuencia Natural $\\omega_n$", f"{wn:.2f} rad/s")
with col2: st.metric("Factor de Amortiguamiento $\\zeta$", f"{zeta:.2f}")

st.markdown("#### **Equivalentes Eléctricos (asumiendo C=1F):**")
st.code(f"R = {R_elec:.3f} Ω | L = {L_elec:.3f} H | C = {C_elec:.1f} F")

# Tiempo
t = np.linspace(0, 15, 1000)

# Gráficas
st.markdown("---")
st.markdown("### 📊 Análisis del Comportamiento")
tab1, tab2, tab3 = st.tabs(["⚡ Respuestas Temporales", "🌀 Polos y Ceros", "📈 Diagrama de Bode"])

with tab1:
    st.subheader("Respuestas al Impulso, Escalón y Rampa (Lazo Abierto vs. Cerrado)")

    fig1, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))

    # 1. Impulso
    t_imp, y_imp_ol = signal.impulse(sys_ol, T=t)
    _, y_imp_lc = signal.impulse(sys_lc, T=t)
    ax1.plot(t_imp, y_imp_ol, 'r', label="Lazo Abierto (OL)"); ax1.plot(t_imp, y_imp_lc, 'r--', label="Lazo Cerrado (CL)")
    ax1.set_title("Respuesta al Impulso"); ax1.set_xlabel("Tiempo (s)"); ax1.grid(True); ax1.legend()

    # 2. Escalón
    t_step, y_step_ol = signal.step(sys_ol, T=t)
    _, y_step_lc = signal.step(sys_lc, T=t)
    ax2.plot(t_step, y_step_ol, 'b', label="Lazo Abierto (OL)"); ax2.plot(t_step, y_step_lc, 'b--', label="Lazo Cerrado (CL)")
    ax2.set_title("Respuesta al Escalón"); ax2.set_xlabel("Tiempo (s)"); ax2.grid(True); ax2.legend()

    # 3. Rampa: G(s)/s
    sys_ramp_ol = signal.TransferFunction(num_ol, np.polymul(den_ol, [1, 0]))
    t_ramp, y_ramp_ol = signal.step(sys_ramp_ol, T=t)

    sys_ramp_lc = signal.TransferFunction(num_lc, np.polymul(den_lc, [1, 0]))
    _, y_ramp_lc = signal.step(sys_ramp_lc, T=t)

    ax3.plot(t_ramp, y_ramp_ol, 'g', label="Lazo Abierto (OL)")
    ax3.plot(t_ramp, y_ramp_lc, 'g--', label="Lazo Cerrado (CL)")
    ax3.plot(t, t, 'k:', label="Rampa Entrada")
    ax3.set_title("Respuesta a la Rampa"); ax3.set_xlabel("Tiempo (s)"); ax3.grid(True); ax3.legend()

    st.pyplot(fig1)

with tab2:
    st.subheader("Mapa de Polos y Ceros")
    fig_pz, ax_pz = plt.subplots()

    # Polos Lazo Abierto
    ax_pz.scatter(np.real(sys_ol.poles), np.imag(sys_ol.poles), marker='x', s=100, color='blue', label='Polos OL')
    # Polos Lazo Cerrado
    ax_pz.scatter(np.real(sys_lc.poles), np.imag(sys_lc.poles), marker='o', s=100, color='red', facecolors='none', edgecolors='red', label='Polos CL')

    ax_pz.axhline(0, color='black', lw=1); ax_pz.axvline(0, color='black', lw=1)
    ax_pz.set_title("Mapa de Polos y Ceros"); ax_pz.set_xlabel("Eje Real"); ax_pz.set_ylabel("Eje Imaginario")
    ax_pz.grid(True); ax_pz.legend()
    st.pyplot(fig_pz)

with tab3:
    st.subheader("Diagrama de Bode (Lazo Abierto vs. Cerrado)")
    w_ol, mag_ol, phase_ol = signal.bode(sys_ol)
    w_lc, mag_lc, phase_lc = signal.bode(sys_lc)

    fig_bode, (ax_m, ax_p) = plt.subplots(2, 1, figsize=(8, 6), sharex=True)

    # Magnitud
    ax_m.semilogx(w_ol, mag_ol, label="Lazo Abierto"); ax_m.semilogx(w_lc, mag_lc, '--', label="Lazo Cerrado")
    ax_m.set_ylabel("Magnitud (dB)"); ax_m.grid(True); ax_m.legend()

    # Fase
    ax_p.semilogx(w_ol, phase_ol, label="Lazo Abierto"); ax_p.semilogx(w_lc, phase_lc, '--', label="Lazo Cerrado")
    ax_p.set_ylabel("Fase (deg)"); ax_p.set_xlabel("Frecuencia (rad/s)"); ax_p.grid(True)
    st.pyplot(fig_bode)

Overwriting pages/2_Sistema.py


In [17]:
# PASO 1: Asegurar que los procesos estén limpios
print("Deteniendo y limpiando procesos anteriores...")
!pkill -f streamlit || true
!pkill -f cloudflared || true

# PASO 2: Lanzar Streamlit y Cloudflared Tunnel
print("Lanzando Streamlit desde 0_Inicio.py...")
!streamlit run 0_Inicio.py &>/content/logs.txt &
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &

# PASO 3: Mostrar URL
import time, re
time.sleep(12)

try:
    with open('/content/cloudflared.log') as f:
        log_data = f.read()
        match = re.search(r"https://[-\w.]+\.trycloudflare\.com", log_data)
        if match:
            print("✅ Tu app está disponible en:", match.group(0))
        else:
            print("❌ No se pudo encontrar el enlace. Intenta ejecutar de nuevo.")
except FileNotFoundError:
    print("❌ Archivo de log no encontrado.")

Deteniendo y limpiando procesos anteriores...
^C
^C
Lanzando Streamlit desde 0_Inicio.py...
✅ Tu app está disponible en: https://nickel-heel-campus-francis.trycloudflare.com
